# Stereo reconstruction in single epoch with 2 different cameras

## Initialization

Let's first set up the python environment by importing the required libraries.

In [1]:
%load_ext autoreload
%autoreload 2

# Import required standard modules
import shutil
import sys
from pathlib import Path

import numpy as np

# Import required icepy4d4D modules
from icepy4d import core as icecore
from icepy4d.core import Epoch, Epoches, EpochDataMap
from icepy4d import matching
from icepy4d import sfm
from icepy4d import io
from icepy4d import utils as icepy4d_utils
from icepy4d.metashape import metashape as MS
from icepy4d.utils import initialization as inizialization

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


First, you have to define the path to the configuration file (`.yaml` file).
This file contains all the paths and parameters needed to run the code.
See the `config.yaml` file in the nootebook folder for an example and refer to the documentation for how to prepare all the data for ICEpy4D.  
Additionally, you can setup a logger for the code to print out some information and a timer to measure the runtime of the code.

In [2]:
# Parse the configuration file
CFG_FILE = "config.yaml"

# Parse the configuration file
cfg_file = Path(CFG_FILE)
cfg = inizialization.parse_cfg(cfg_file, ignore_errors=True)

# Initialize the logger
logger = icepy4d_utils.get_logger()

# Initialize a timer to measure the processing time
timer = icepy4d_utils.AverageTimer()

# Get the list of cameras from the configuration file
cams = cfg.cams


ICEpy4D
Image-based Continuos monitoring of glaciers' Evolution with low-cost stereo-cameras and Deep Learning photogrammetry
2023 - Francesco Ioli - francesco.ioli@polimi.it

2023-09-19 10:33:43 | [INFO    ] Configuration file: config.yaml
2023-09-19 10:33:43 | [INFO    ] Epoch_to_process set to a pair of values. Expanding it for a range of epoches from epoch 0 to 158.


## ICEpy4D main structures

For the processing, you have to initialize all the required variables. This procedure is the same also for multi-epoch processing.


### EpochDataMap
The `EpochDataMap` class is a critical structure for multi-epoch processing in ICEpy4D. It helps organize and manage data for each epoch, including timestamps, associated images, and time differences.
`EpochDataMap` is a dictionary that contains the information the timestamp (i.e., date and time) of each epoch and the images belonging to that epoch.
The purpose of this class is to give a east-to-use tool for associating the timestamp of each epoch to the images belonging to that epoch.

The `EpochDataMap` allows for automatically define the association of the images to the epochs, based on the timestamp of the images. 
The images are associated to the epoch with the closest timestamp. The `EpochDataMap` first select the master camera (by default, the first camera). Then, for each image of the master camera, it looks for the closest image taken by all the other cameras (i.e., named 'slave cameras') based on their timestamp. As the cameras may not be perfectly syncronized, a time tolerance can be defined to allow for a maximum time difference between the images of different cameras (by default, 180 seconds). If the time difference between a slave image and the master image is larger than the time tolerance, the slave image is not associated to the current epoch.
It may be possible that one epoch contains only the image of the master camera (e.g., due to a disruption of the other cameras during that day), therefore a minimum number of images can be defined for an epoch to be included in the `EpochDataMap` (by default, 2 images). If less than the minimum number of images are associated to an epoch, the epoch is discarded.

1. The EpochDataMap structure

    The key of the dictionary is the epoch number (an integer number, starting from 0). The values of the dictionary are the `EpochData` class are again dictionaries that contains the timestamp of the epoch, the images associated at that epoch and the time difference between the timestamp of each camera and that of the master camera (by default, the first camera).
    For instance, a value of the `EpochDataMap` dictionary is the following:

    ```python
    epoch_data_map[0] = {
        'timestamp': datetime.datetime(2023, 6, 23, 9, 59, 58), # timestamp of the first epoch
        'images': { 
            'p1': Image data/img/p1/p1_20230623_095958_IMG_0845.JPG,
            'p2': Image data/img/p2/p2_20230623_100000_IMG_0582.JPG
            }, # images of the first epoch
        'dt': {'p1': datetime.timedelta(0), 'p2': datetime.timedelta(seconds=2)} # time difference between each camera and the master camera
    }
    ```

    For accessing the data inside the dictionary for each epoch, you can use the `dot notation` for getting the timestamp, the image dictionary and the time differences.

    ``` python
    epoch_map[0].timestamp 
    epoch_map[0].images 
    epoch_map[0].dt 
    ```

    The epoch timestamp (`epoch_map[n].timestamp`) is taken from the timestamp of the master camera, and it is stored as a datetime object.

    ``` 
    epoch_map[0].timestamp = datetime.datetime(2023, 6, 23, 9, 59, 58)
    ```

    The images of each epoch is again a dictionary with the camera names (i.e., the name of the folder containing the image sequence) as keys and Image objects as values.

    ```python
    epoch_map[0].images['p1'] = Image("data/img/p1/p1_20230623_095958_IMG_0845.JPG")
    epoch_map[0].images['p2'] = Image("data/img/p2/p2_20230623_100000_IMG_0582.JPG")
    ```


2. How to inizialize a EpochDataMap

    Initialize the `EpochDataMap` object by providing the image directory and optional parameters like the time tolerance (maximum time difference allowed between images from different cameras) and minimum number of images required for an epoch to be included.

    ```python
    epoch_map = EpochDataMap('path_to_image_directory', time_tolerance_sec=180, min_images=2)
    ```

    If not specified, the time tolerance is set to 180 seconds and the minimum number of images is set to 2.

In [3]:
# Build the EpochDataMap object find pairs of coheval images for each epoch
epoch_map = EpochDataMap(cfg.paths.image_dir, time_tolerance_sec=1200)

print(f"Epoch 0  Timestamp: {epoch_map[0].timestamp}")
print(f"\t Images: {epoch_map[0].images}")
print(f"\t Delta t from master camera (Cam 0): {epoch_map[0].dt}")

2023-09-19 10:33:58 | [INFO    ] Building EpochDataMap: found 158 epochs
2023-09-19 10:33:58 | [INFO    ] Mean max dt: 590.37 seconds (max: 709.00 seconds))
2023-09-19 10:33:58 | [INFO    ] Removed 0 epochs with less than 2 images
Epoch 0  Timestamp: 2022-05-01 14:01:15
	 Images: {'p1': Image /home/francesco/phd/icepy4d/notebooks/../data/img/p1/IMG_2637.jpg, 'p2': Image /home/francesco/phd/icepy4d/notebooks/../data/img/p2/IMG_1112.jpg}
	 Delta t from master camera (Cam 0): {'p1': datetime.timedelta(0), 'p2': datetime.timedelta(seconds=464)}


### Epoch

Another crucial structure for multi-epoch processing is the `Epoch` class. It all the data belonging to each epoch and i

#### Create a new Epoch object

In [9]:
# Set id to process
ep = cfg.proc.epoch_to_process[0]

# Define paths to the epoch directory
epoch_name = epoch_map.get_timestamp_str(ep)
epochdir = cfg.paths.results_dir / epoch_name

# Get the list of images for the current epoch
im_epoch = epoch_map.get_images(ep)

# Load cameras
cams_ep = {}
for cam in cams:
    calib = icecore.Calibration(cfg.paths.calibration_dir / f"{cam}.txt")
    cams_ep[cam] = calib.to_camera()

# Load targets
target_paths = [
    cfg.georef.target_dir
    / (im_epoch[cam].stem + cfg.georef.target_file_ext)
    for cam in cams
]
targ_ep = icecore.Targets(
    im_file_path=target_paths,
    obj_file_path=cfg.georef.target_dir
    / cfg.georef.target_world_file,
)

# Create empty features
feat_ep = {cam: icecore.Features() for cam in cams}

# Create the epoch object
epoch = Epoch(
    timestamp=epoch_map.get_timestamp_str(ep),
    images=im_epoch,
    cameras=cams_ep,
    features=feat_ep,
    targets=targ_ep,
    epoch_dir=epochdir,
)
print(f"Epoch: {epoch}")

Epoch: 2022-05-01_14-01-15


## Stereo Processing
The stereo processing is carried out for each epoch in order to find matched features, estimating camera pose, and triangulating the 3D points. 
The output of this step is a set of 3D points and their corresponding descriptors.

The processing for all the epoches is then iterated in a big loop.

#### Feature matching with SuperGlue

Wide-baseline feature matching is performed using the SuperGlue algorithm.
Refer to the `matching.ipynb` notebook for more details about the matching process and explanation of the parameters.

In [ ]:
# Define matching parameters
matching_quality = matching.Quality.HIGH
tile_selection = matching.TileSelection.PRESELECTION
tiling_grid = [4, 3]
tiling_overlap = 200
geometric_verification = matching.GeometricVerification.PYDEGENSAC
geometric_verification_threshold = 1
geometric_verification_confidence = 0.9999
match_dir = epoch.epoch_dir / "matching"

# Create a new matcher object
matcher = matching.SuperGlueMatcher(cfg.matching)
matcher.match(
    epoch.images[cams[0]].value,
    epoch.images[cams[1]].value,
    quality=matching_quality,
    tile_selection=tile_selection,
    grid=tiling_grid,
    overlap=tiling_overlap,
    do_viz_matches=True,
    do_viz_tiles=False,
    save_dir=match_dir,
    geometric_verification=geometric_verification,
    threshold=geometric_verification_threshold,
    confidence=geometric_verification_confidence,
)
timer.update("matching")

Extract the matched features from the Matcher object and save them in the current Epoch object

In [ ]:
# Define a dictionary with empty Features objects for each camera, which will be filled with the matched keypoints, descriptors and scores
f = {cam: icecore.Features() for cam in cams}

# Stack matched keypoints, descriptors and scores into Features objects
f[cams[0]].append_features_from_numpy(
    x=matcher.mkpts0[:, 0],
    y=matcher.mkpts0[:, 1],
    descr=matcher.descriptors0,
    scores=matcher.scores0,
)
f[cams[1]].append_features_from_numpy(
    x=matcher.mkpts1[:, 0],
    y=matcher.mkpts1[:, 1],
    descr=matcher.descriptors1,
    scores=matcher.scores1,
)

# Store the dictionary with the features in the Epoch object
epoch.features = f

#### Scene reconstruction

First, perform Relative orientation of the two cameras by using the matched features and the a-priori camera interior orientation.

In [ ]:
# Initialize RelativeOrientation class with a list containing the two
# cameras and a list contaning the matched features location on each camera.
relative_ori = sfm.RelativeOrientation(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
relative_ori.estimate_pose(
    threshold=cfg.matching.pydegensac_threshold,
    confidence=0.999999,
    scale_factor=np.linalg.norm(
        cfg.georef.camera_centers_world[0] - cfg.georef.camera_centers_world[1]
    ),
)
# Store result in camera 1 object
epoch.cameras[cams[1]] = relative_ori.cameras[1]

cfg.georef.camera_centers_world

In [ ]:
camera_baseline = np.linalg.norm(
        cfg.georef.camera_centers_world[0] - cfg.georef.camera_centers_world[1]
    )
image = epoch.images[cams[0]].value

In [ ]:
# Relative orientation
feature0 = epoch.features[cams[0]].kpts_to_numpy()
feature1 = epoch.features[cams[1]].kpts_to_numpy()

relative_ori = sfm.RelativeOrientation(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [feature0, feature1],
)
relative_ori.estimate_pose(scale_factor=camera_baseline)
epoch.cameras[cams[1]] = relative_ori.cameras[1]

# Triangulation
triang = sfm.Triangulate(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [feature0, feature1],
)
points3d = triang.triangulate_two_views(
    compute_colors=True, image=image, cam_id=1
)

Triangulate points into the object space

In [ ]:
triang = sfm.Triangulate(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
points3d = triang.triangulate_two_views(
    compute_colors=True, image=images[cams[1]].read_image(ep).value, cam_id=1
)

# Update timer
timer.update("triangulation")

Perform an absolute orientation of the current solution (i.e., cameras' exterior orientation and 3D points) by using the ground control points.

The coordinates of the two cameras are used as additional ground control points for estimating a Helmert transformation.

In [ ]:
# Get targets available in all cameras. The Labels of valid targets are returned as second element by the get_image_coor_by_label() method
valid_targets = epoch.targets.get_image_coor_by_label(
    cfg.georef.targets_to_use, cam_id=0
)[1]

# Check if the same targets are available in all cameras
for id in range(1, len(cams)):
    assert (
        valid_targets
        == epoch.targets.get_image_coor_by_label(
            cfg.georef.targets_to_use, cam_id=id
        )[1]
    ), f"""epoch {ep} - {epoch_map.get_timestamp(ep)}: 
    Different targets found in image {id} - {images[cams[id]][ep]}"""

# Check if there are enough targets
assert len(valid_targets) > 1, f"Not enough targets found in epoch {ep}"

# If not all the targets defined in the config file are found, log a warning and use only the valid targets
if valid_targets != cfg.georef.targets_to_use:
    logger.warning(f"Not all targets found. Using onlys {valid_targets}")

# Get image and object coordinates of valid targets
image_coords = [
    epoch.targets.get_image_coor_by_label(valid_targets, cam_id=id)[0]
    for id, cam in enumerate(cams)
]
obj_coords = epoch.targets.get_object_coor_by_label(valid_targets)[0]

# Perform absolute orientation
abs_ori = sfm.Absolute_orientation(
    (epoch.cameras[cams[0]], epoch.cameras[cams[1]]),
    points3d_final=obj_coords,
    image_points=image_coords,
    camera_centers_world=cfg.georef.camera_centers_world,
)
T = abs_ori.estimate_transformation_linear(estimate_scale=True)
points3d = abs_ori.apply_transformation(points3d=points3d)
for i, cam in enumerate(cams):
    epoch.cameras[cam] = abs_ori.cameras[i]

# Convert the 3D points to an icepy4d Points object
pts = icecore.Points()
pts.append_points_from_numpy(
    points3d,
    track_ids=epoch.features[cams[0]].get_track_ids(),
    colors=triang.colors,
)

# Store the points in the Epoch object
epoch.points = pts

# Update timer
timer.update("absolute orientation")

Save the current Epoch object as a pickle file.

In [ ]:
# Save epoch as a pickle object
if epoch.save_pickle(f"{epoch.epoch_dir}/{epoch}.pickle"):
    logger.info(f"{epoch} saved successfully")
else:
    logger.error(f"Unable to save {epoch}")